In [1]:

import cv2 #importing opencv library for video and image tools
import numpy as np #NumPy is a general-purpose array-processing package.
#It provides a high-performance multidimensional array object, and tools for working with these arrays.
import os #provides function for interacting with operating system
from IPython.display import display, clear_output #for various types of display


In [2]:
#haarcascade is an Algorithm which is used for detection of faces.It is comapritively slower as compared to LBHP Algorithm but has more accuracy.
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
#function to find squareroot of the sum of squares of differences
def dist(x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))

In [4]:
#function to predict the label closest to their training data set using k nearest neighbours
#X here contains the encoded face data 
#Y here stores the distance values of  the query point(detected face) and the training dataset
#k defines the nearest neighbours
def knn(X,Y,queryPoint,k=5): 
    vals = [] #vals here stores the distance values of  the query point(detected face) and the training dataset
    m = X.shape[0]
    for i in range(m):
        d = dist(queryPoint,X[i]) 
        vals.append((d,Y[i])) 
    vals = sorted(vals) #sorting the vals array to determine the nearest/first k points
     
    vals = vals[:k]
    vals = np.array(vals) #converting it to numpy array
    new_vals = np.unique(vals[:,1],return_counts=True) #checking for unique values
    index = new_vals[1].argmax() #returns the index of maximum element of a particular axis 
    pred = new_vals[0][index] #finding the predicted value using the index value
    return pred

In [5]:
#function to collect the training dataset
def train():
    cap = cv2.VideoCapture(0) #opening webcamera
    skip = 0
    photos_clicked = [] #array to store the photos of an individual
    dataset_path = 'images/' #creating a folder to store images
    file_name = input("Enter your name: ") #taking input from user
    while True: 
        skip += 1
        ret,frame = cap.read() 
        if ret == False:
            continue
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #converting to gray image as open cv accepts gray images
        faces = face_cascade.detectMultiScale(frame,1.3,5)  #using haarcascade algorithm to detect faces
        faces = sorted(faces,key=lambda f:f[2]*f[3]) #sorting the faces based on their distance from the camera
        # Pick the last face as it will be the largest
        for face in faces[-1:]: 
            x,y,w,h = face
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2) #drawing a box around the face
        # Extract the region of interest
            offset = 10
            face_section = gray_frame[y-offset:y+h+offset,x-offset:x+w+offset] #since we have to find the facial features we need to find region of interest, thereby cropping to get face encodings
            face_section = cv2.resize(face_section,(100,100)) #resizing the face section
            if skip%10 == 0: #after 10 frames the image is captured
                photos_clicked.append(face_section) #they are added to the face_section array 
                print(len(photos_clicked))#prints the images being saved
        cv2.imshow("Frame",frame) #continuously showing the webcamera frame
        key_pressed = cv2.waitKey(1) & 0xFF #to quit the entire process press "q" on the webcam screen
        if key_pressed == ord('q'):
            break

    photos_clicked = np.asarray(photos_clicked) # Convert our face list array to a numpy array
    photos_clicked = photos_clicked.reshape((photos_clicked.shape[0],-1)) #reshaping to suit the format of  numpy arrays
    print(photos_clicked.shape)#print the number of photos saved

    np.save(dataset_path+file_name+'.npy',photos_clicked) # Save this data to the file system
    print("Images saved successfully!")
    cap.release()#releasing the captured webcam frame
    cv2.destroyAllWindows() #destroying all frames

In [6]:
font = cv2.FONT_HERSHEY_DUPLEX

def test():
    cap = cv2.VideoCapture(0) #opening webcamera
    
    dataset_path = 'images/'
    face_data = [] #array to store face_data
    labels = [] #array to store labels mapping between id and names
    names={} #array to store names 
    class_id = 0
    for fx in os.listdir(dataset_path): #here fx stands for the files with each individual's images saved as npy
        if fx.endswith('.npy'):
            data_item = np.load(dataset_path+fx) #loading the file
            face_data.append(data_item) #append in the face_data
       
            target = class_id*np.ones((data_item.shape[0],))  # Create labels for the class
            labels.append(target) #adding to labels
            names[class_id] = fx[:-4] #storing the names of individual
            class_id += 1
    face_data = np.concatenate(face_data,axis=0) #concatenating facedata of every individual 
    labels = np.concatenate(labels,axis=0)#concatenating label of every individual 
    test_point = np.zeros((100,100))  
    while True:
        clear_output(wait=True)
        try:
            ret,frame = cap.read() 
            if ret == False:
                continue
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(frame,1.3,5)
            faces = sorted(faces,key=lambda f:f[2]*f[3])
            # Pick the last face as it will be the largest
            faces_in_frame=[]
            for face in faces:
                x,y,w,h = face
                l=x
                b=y
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
                # Extract the region of interest
                offset = 10
                face_section = gray_frame[y-offset:y+h+offset,x-offset:x+w+offset]
                test_point = cv2.resize(face_section,(100,100))


                y = test_point.reshape((-1,)) #query point ->the face for which we want to identify
                label = knn(face_data,labels,y) #getting the label from the K-nn function
                faces_in_frame.append(names[int(label)]) #appending all the faces in the frames
                cv2.rectangle(frame, (l, b - 25), (l+w, b), (0, 0, 255), cv2.FILLED) #drawing a rectangle for displaying the name
                cv2.putText(frame, names[int(label)], (l + 6, b - 6), font, 0.5, (255, 255, 255), 1) #putting text in the rectangle formed above

            print("Hello "+' '.join(faces_in_frame)) #displaying all the faces in frames
            cv2.imshow("Frame",frame) #showing the frame
            if cv2.waitKey(1) & 0xFF == ord('q'): #waiting for the frame window
                break
        except:
            cv2.imshow("Frame",frame) #if no faces detected in the frame the error is caught in this section 
            print("Code not working")
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            #cv2.waitKey(0) & 0xFF
            

    cap.release() #releasing the webcam
    cv2.destroyAllWindows() #closing all windows

In [7]:
train()

Enter your name: Saanidhi


error: /opt/concourse/worker/volumes/live/d8bcd4d1-79b2-4aa5-797a-b95097f1118f/volume/opencv_1512680501887/work/modules/objdetect/src/cascadedetect.cpp:1698: error: (-215) !empty() in function detectMultiScale


In [8]:
test()

FileNotFoundError: [Errno 2] No such file or directory: 'images/'

In [2]:
    cap = cv2.VideoCapture(0)
